In [1]:
from tensorflow.keras.layers import Input, Dense, Softmax, ReLU
from tensorflow.keras import Model
import numpy as np
import tensorflow as tf

2024-02-02 17:54:05.106266: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 17:54:05.126623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 17:54:05.126644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 17:54:05.127152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 17:54:05.130406: I tensorflow/core/platform/cpu_feature_guar

In [2]:
input_size = 14 * 14
layer_1 = 25
n_class = 10

In [12]:
inputs = Input(shape=(input_size,))
out = Dense(layer_1)(inputs)
out = ReLU()(out) # not support dor dense layer
out = Dense(n_class)(out)
#out = Softmax()(out)
c_aware_model = Model(inputs, out)

### Convert to Circom Circuit

In [4]:

# model_in_path = model_path + arch_folder + model_name
# model_tf = tf.keras.models.load_model(model_in_path)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense (Dense)               (None, 25)                4925      
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

313/313 - 0s - loss: 2.4390 - accuracy: 0.1188 - 287ms/epoch - 917us/step

Test accuracy: 0.11879999935626984


2024-02-01 15:51:16.580278: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 15:51:16.599602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 15:51:16.599623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 15:51:16.600112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 15:51:16.603144: I tensorflow/core/platform/cpu_feature_guar

CompletedProcess(args=['python', 'main.py', 'c_aware.h5', '-o./circuits/golden.circom'], returncode=0)

## 196_25_10

In [388]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
#X = test_images_tf[4]
X = np.random.rand(1,25)

X_in = [int(x*1e36) for x in X[0]]

In [500]:
inputs = Input(shape=(196,))
out = Dense(25)(inputs)
out = ReLU()(out) # not support dor dense layer
out = Dense(10)(out)
#out = Softmax()(out)
simple_model = Model(inputs, out)

simple_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

simple_model.summary()

Model: "model_133"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_96 (InputLayer)       [(None, 196)]             0         
                                                                 
 dense_237 (Dense)           (None, 25)                4925      
                                                                 
 re_lu_118 (ReLU)            (None, 25)                0         
                                                                 
 dense_238 (Dense)           (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [311]:
simple_model.save('simple.h5')

circuit_folder = "./circuits"

command = ['python','main.py', 'simple.h5', f"-o{circuit_folder}/simple.circom"]
subprocess.run(command)

/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024-02-01 16:46:16.174242: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 16:46:16.194285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 16:46:16.194309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plu

CompletedProcess(args=['python', 'main.py', 'simple.h5', '-o./circuits/simple.circom'], returncode=0)

In [442]:
model_tf.summary()

Model: "model_73"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, 784)]             0         
                                                                 
 dense_70 (Dense)            (None, 56)                43960     
                                                                 
 re_lu_10 (ReLU)             (None, 56)                0         
                                                                 
 dense_71 (Dense)            (None, 10)                570       
                                                                 
Total params: 44530 (173.95 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [441]:
simple_model.summary()

Model: "model_74"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_38 (InputLayer)       [(None, 25)]              0         
                                                                 
 dense_72 (Dense)            (None, 10)                260       
                                                                 
 re_lu_11 (ReLU)             (None, 10)                0         
                                                                 
 dense_73 (Dense)            (None, 4)                 44        
                                                                 
Total params: 304 (1.19 KB)
Trainable params: 304 (1.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [439]:
def transfer_weights(layers, model, scalar = 36):
    weights = []
    biases = []
    for ind in range(len(layers)-1):
        w = [int(model.weights[ind * 2].numpy()[i][j]*10**scalar) for j in range(layers[ind+1]) for i in range(layers[ind])]
        b = [int(model.weights[ind * 2 + 1].numpy()[i]*10**(scalar * 2)) for i in range(layers[ind+1])]
        weights.append(w)
        biases.append(b)

    return weights, biases


In [440]:
weights, biases = transfer_weights([25,10,4], simple_model)

In [389]:
Dense32weights = [[int(simple_model.weights[0].numpy()[i][j]*1e36) for j in range(10)] for i in range(25)]
Dense32bias = [int(simple_model.weights[1].numpy()[i]*1e72) for i in range(10)]

Dense21weights = [[int(simple_model.weights[2].numpy()[i][j]*1e36) for j in range(4)] for i in range(10)]
Dense21bias = [int(simple_model.weights[3].numpy()[i]*1e72) for i in range(4)]

In [390]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]

    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
        
    return Input, Weights, Bias, out, remainder

In [ ]:
def update_input_json(layers, weights, biases, x_in, scalar=36):
    dense_weights = []
    dense_biases = []
    dense_out = []
    dense_remainder = []
    xin = x_in
    for ind in range(len(layers)-1):
        _, w, b, out, rem = DenseInt(layers[ind*2], layers[ind*2+1], 10 ** scalar, )

In [391]:
X_in, Dense32weights, Dense32bias, Dense32out, Dense32remainder = DenseInt(25,10, 10**36, X_in, Dense32weights, Dense32bias)
Dense32out

['21888242871839275222246405745257275088547889099137421933185887936077399578990',
 '21888242871839275222246405745257275088547891392005500877761426869380115728365',
 '21888242871839275222246405745257275088547461870640440768395816271950941849225',
 '760882669958289614497683152132671299',
 '375355821196145534246890766214109674',
 '21888242871839275222246405745257275088547755232809817291404017824716958324549',
 '367232317750539579745806675835950719',
 '21888242871839275222246405745257275088547536954470377675163383772209904425022',
 '41789005957338798850275320509232227',
 '21888242871839275222246405745257275088547993031167967781180782225408529842975']

In [392]:
layer_output = simple_model.layers[1].output

# Create a new model that gives the output of the last dense layer
intermediate_model = tf.keras.Model(inputs=simple_model.input, outputs=layer_output)

intermediate_model.predict(X)

1/1 [==============================] - 0s 27ms/step


array([[-0.4753013 , -0.47300842, -0.9025297 ,  0.7608827 ,  0.37535578,
        -0.6091676 ,  0.36723226, -0.827446  ,  0.041789  , -0.3713692 ]],
      dtype=float32)

In [393]:
ReLUout = [Dense32out[i] if int(Dense32out[i]) < p//2 else 0 for i in range(10)]
Dense21in = [int(ReLUout[i]) for i in range(10)]

In [394]:
_, Dense21weights, Dense21bias, Dense21out, Dense21remainder = DenseInt(10, 4, 10**36, Dense21in, Dense21weights, Dense21bias)
Dense21out, Dense21remainder

(['21888242871839275222246405745257275088548226105548729866612023329782859643251',
  '45774850344441100328034339654364161',
  '21888242871839275222246405745257275088547866230673650181312964938000640985424',
  '21888242871839275222246405745257275088548136813565464080727960045542101572962'],
 ['478871644325988034016738740847771648',
  '982215161144976163118173428595556352',
  '656651603526482482717129219032219648',
  '938080816682802168463026458208501760'])

In [395]:
layer_output = simple_model.layers[3].output

# Create a new model that gives the output of the last dense layer
intermediate_model = tf.keras.Model(inputs=simple_model.input, outputs=layer_output)

intermediate_model.predict(X) 

1/1 [==============================] - 0s 21ms/step


array([[-0.13829488,  0.04577482, -0.49816972, -0.2275868 ]],
      dtype=float32)

In [396]:
a = [int(Dense21out[i]) if int(Dense21out[i]) < p//2 else 0 for i in range(4)]
a

[0, 45774850344441100328034339654364161, 0, 0]

In [397]:
np.argmax(a)

1

In [398]:
np.argmax(simple_model.predict(X))

1/1 [==============================] - 0s 32ms/step


1

In [399]:
import json

in_json = {
    "in": X_in,
    "Dense32weights": Dense32weights,
    "Dense32bias": Dense32bias,
    "Dense32out": Dense32out,
    "Dense32remainder": Dense32remainder,
    "ReLUout": ReLUout,
    "Dense21weights": Dense21weights,
    "Dense21bias": Dense21bias,
    "Dense21out": Dense21out,
    "Dense21remainder": Dense21remainder
}


with open("./test_input.json", "w") as f:
    json.dump(in_json, f)

## Proof Generation and Verification

In [410]:
import psutil, time, threading
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    print(f"Maximum memory used: {max_memory} MB")
    return max_memory

def run_bash(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Get the process ID
    pid = process.pid
    print(f"Process ID: {pid}")

    # Start memory monitoring in a separate thread
    monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
    monitor_thread.start()

    # Wait for the process to complete and capture output
    stdout, stderr = process.communicate()

    # Wait for the monitoring thread to finish
    monitor_thread.join()
    return stdout, stderr

In [402]:
# Compile
target_circom = "./circuits/test.circom"
command = ['circom', target_circom, "--r1cs", "--wasm", "--sym", "--c"]
subprocess.run(command)

template instances: 15
non-linear constraints: 5460
linear constraints: 0
public inputs: 0
private inputs: 367 (353 belong to witness)
public outputs: 4
wires: 5774
labels: 13127
Written successfully: ./test.r1cs
Written successfully: ./test.sym
Written successfully: ./test_cpp/test.cpp and ./test_cpp/test.dat
Written successfully: ./test_cpp/main.cpp, circom.hpp, calcwit.hpp, calcwit.cpp, fr.hpp, fr.cpp, fr.asm and Makefile
Written successfully: ./test_js/test.wasm
Everything went okay


CompletedProcess(args=['circom', './circuits/test.circom', '--r1cs', '--wasm', '--sym', '--c'], returncode=0)

In [403]:
# Witness
gen_wit_path = "./test_js/generate_witness.js"
wasm_path = "./test_js/test.wasm"
input_path = "test_input.json"

command = ['node', gen_wit_path, wasm_path, input_path, "witness.wtns" ]
subprocess.run(command)


CompletedProcess(args=['node', './test_js/generate_witness.js', './test_js/test.wasm', 'test_input.json', 'witness.wtns'], returncode=0)

### Power of Tau

In [ ]:
2

In [430]:
# Setup
digit = 14
command = ['snarkjs', 'powersoftau', 'new', 'bn128', str(digit), './ceremony/pot12_0000.ptau','-v']
subprocess.run(command)

[DEBUG] snarkJS: Calculating First Challenge Hash
[DEBUG] snarkJS: Calculate Initial Hash: tauG1
[DEBUG] snarkJS: Calculate Initial Hash: tauG2
[DEBUG] snarkJS: Calculate Initial Hash: alphaTauG1
[DEBUG] snarkJS: Calculate Initial Hash: betaTauG1
[DEBUG] snarkJS: Blank Contribution Hash:
		786a02f7 42015903 c6c6fd85 2552d272
		912f4740 e1584761 8a86e217 f71f5419
		d25e1031 afee5853 13896444 934eb04b
		903a685b 1448b755 d56f701a fe9be2ce
[INFO]  snarkJS: First Contribution Hash:
		bc0bde79 80381fa6 42b20975 91dd83f1
		ed15b003 e15c3552 0af32c95 eb519149
		2a6f3175 215635cf c10e6098 e2c612d0
		ca84f1a9 f90b5333 560c8af5 9b9209f4


CompletedProcess(args=['snarkjs', 'powersoftau', 'new', 'bn128', '14', './ceremony/pot12_0000.ptau', '-v'], returncode=0)

In [426]:
# Contribute
!snarkjs powersoftau contribute pot12_0000.ptau pot12_0001.ptau --name="1st" -v

Enter a random text. (Entropy): 

In [431]:
# Phase 2
command = ['snarkjs', 'powersoftau', 'prepare', 'phase2', './ceremony/pot12_0001.ptau','./ceremony/pot12_final.ptau', '-v']
subprocess.run(command)
#stdout, stderr = run_bash(command)
#stdout, stderr

[DEBUG] snarkJS: Starting section: tauG1
[DEBUG] snarkJS: tauG1: fft 0 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 0 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 4 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix start: 1/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 1/2
[DEBUG] snarkJS: tauG1: fft  4  join: 4/4
[DEBUG] snarkJS: tauG1: fft 4 join  4/4  1/1 0/1
[DEBUG] snarkJS: tauG1: fft 5 mix start: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 2/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 3/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 2/4
[DEBUG] snarkJS: tauG1: fft 5 m

CompletedProcess(args=['snarkjs', 'powersoftau', 'prepare', 'phase2', './ceremony/pot12_0001.ptau', './ceremony/pot12_final.ptau', '-v'], returncode=0)

In [432]:
command = ['snarkjs', 'groth16', 'setup', 'test.r1cs', './ceremony/pot12_final.ptau', './ceremony/test_0000.zkey']
subprocess.run(command)

[INFO]  snarkJS: Reading r1cs
[INFO]  snarkJS: Reading tauG1
[INFO]  snarkJS: Reading tauG2
[INFO]  snarkJS: Reading alphatauG1
[INFO]  snarkJS: Reading betatauG1
[INFO]  snarkJS: Circuit hash: 
		6859a61f 99c60ec7 dd112c19 663d7725
		d65e2be9 1641a7a6 01337305 3828c0ef
		ce32d971 497f9b78 5be17e57 eed7f23b
		ab83769d 9dc6ed23 b64832c0 0dbbd722


CompletedProcess(args=['snarkjs', 'groth16', 'setup', 'test.r1cs', './ceremony/pot12_final.ptau', './ceremony/test_0000.zkey'], returncode=0)

In [ ]:
!snarkjs zkey contribute ./ceremony/test_0000.zkey ./ceremony/test_0001.zkey --name="1st" -v


In [435]:
command = ['snarkjs', 'zkey', 'export','verificationkey', './ceremony/test_0001.zkey','./ceremony/verification_key.json']
subprocess.run(command)


CompletedProcess(args=['snarkjs', 'zkey', 'export', 'verificationkey', './ceremony/test_0001.zkey', './ceremony/verification_key.json'], returncode=0)

### Proof!

In [436]:
command = ['snarkjs', 'groth16', 'prove','./ceremony/test_0001.zkey', 'witness.wtns', 'proof.json', 'public.json']
subprocess.run(command)


CompletedProcess(args=['snarkjs', 'groth16', 'prove', './ceremony/test_0001.zkey', 'witness.wtns', 'proof.json', 'public.json'], returncode=0)

In [437]:
command = ['snarkjs', 'groth16', 'verify','./ceremony/verification_key.json', 'public.json', 'proof.json']
subprocess.run(command)


[INFO]  snarkJS: OK!


CompletedProcess(args=['snarkjs', 'groth16', 'verify', './ceremony/verification_key.json', 'public.json', 'proof.json'], returncode=0)

## 784_56_10

In [381]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
#X = test_images_tf[4]
X = np.random.rand(1,784)[0]

X_in = [int(x*1e36) for x in X]

In [363]:
inputs = Input(shape=(28*28,))
out = Dense(56)(inputs)
out = ReLU()(out) # not support dor dense layer
out = Dense(10)(out)
#out = Softmax()(out)
model_tf = Model(inputs, out)

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [373]:

Dense32weights = [[int(model_tf.weights[0].numpy()[i][j]*1e36) for j in range(56)] for i in range(784)]
Dense32bias = [int(model_tf.weights[1].numpy()[i]*1e72) for i in range(56)]

Dense21weights = [[int(model_tf.weights[2].numpy()[i][j]*1e36) for j in range(10)] for i in range(56)]
Dense21bias = [int(model_tf.weights[3].numpy()[i]*1e72) for i in range(10)]

In [374]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder

In [375]:
X_in, Dense32weights, Dense32bias, Dense32out, Dense32remainder = DenseInt(784, 56, 10**36, X_in, Dense32weights, Dense32bias)
Dense32out, Dense32remainder

(['599844176058507500715870227047794144',
  '254085832562279814842563455586042834',
  '21888242871839275222246405745257275088548098151932762640080048754692153991938',
  '21888242871839275222246405745257275088548054101938761120920236178326078324920',
  '21888242871839275222246405745257275088546991560831278405601752448020019551205',
  '350965177824582374428618857588000159',
  '21888242871839275222246405745257275088548247273076989580935732299107334852663',
  '21888242871839275222246405745257275088547892075237087304948556684022153044542',
  '1523689990253232348920477713363976126',
  '64269576800380251297741112525401846',
  '21888242871839275222246405745257275088547617302151967562938309904862738478447',
  '21888242871839275222246405745257275088548080884821766432362687733706738825483',
  '681990296234564654537531598091934491',
  '308547500079025032049038299281146298',
  '21888242871839275222246405745257275088547968620717650928324986289973294287951',
  '218882428718392752222464057452572750885

In [376]:
ReLUout = [Dense32out[i] if int(Dense32out[i]) < p//2 else 0 for i in range(56)]
Dense21in = [int(ReLUout[i]) for i in range(56)]

In [377]:
_, Dense21weights, Dense21bias, Dense21out, Dense21remainder = DenseInt(56, 10, 10**36, Dense21in, Dense21weights, Dense21bias)
Dense21out, Dense21remainder

(['21888242871839275222246405745257275088548255588336183010354998461476555955752',
  '122371314147728789627360970109761745',
  '469902678785596500354676862598940319',
  '173337373777368768382616647181246730',
  '21888242871839275222246405745257275088548322624754541161612379539651634713582',
  '106313702541773808720209063629280936',
  '438786513675779915297221109273639753',
  '21888242871839275222246405745257275088547511607844254465561510259833697385740',
  '21888242871839275222246405745257275088546857539239101898142972822308825717676',
  '156261022323811374000412351885344068'],
 ['44848485296297564982385474097119232',
  '161069084926350677236207370966663168',
  '895648943938878146220929297302945792',
  '506232756495429178206315085303906304',
  '3234056260405296681922087220150272',
  '518816313488281093224651873314668544',
  '281569286973552914018596598354804736',
  '520372580488832076198868913974411264',
  '662657668639482397598247625701720064',
  '370344562959898518919074102862938112'

In [378]:
import json
in_json = {
    "in": X_in,
    "Dense32weights": Dense32weights,
    "Dense32bias": Dense32bias,
    "Dense32out": Dense32out,
    "Dense32remainder": Dense32remainder,
    "ReLUout": ReLUout,
    "Dense21weights": Dense21weights,
    "Dense21bias": Dense21bias,
    "Dense21out": Dense21out,
    "Dense21remainder": Dense21remainder
}

with open("dnn_input.json", "w") as f:
    json.dump(in_json, f)